In [56]:
%reload_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# Часть 1

ВЫ ПРИНЯЛИ ВСЁ, КРОМЕ 2ой, Т.К. ТАМ НЕ БЫЛО ПРОВЕРКИ НА СОТРУДНИКА МАЙКРОСОФТ И Я ДОБАВИЛ 10ую. ВРОДЕ БЫ ВСЁ РАБОТАЕТ КАК НАДО. НУ И УБРАЛ ВЕЗДЕ ЛИМИТ, КРОМЕ 1ой

## Задание 1

Вывести 20 самых больших по размеру треков жанра ROCK и формата MPEG (используйте команду LIMIT)

In [57]:
%%sql 
SELECT
      tracks.Name
FROM
      tracks 
LEFT JOIN ( 
            SELECT genres.Name        as GenreName,
                   media_types.Name   as MediaTypeName,
                   * 
            FROM genres, media_types) as Genres_and_mediaType USING ( GenreId, MediaTypeId )
WHERE 
      GenreName     = 'Rock' AND 
      MediaTypeName = 'MPEG audio file'
ORDER BY
        Bytes DESC
LIMIT 20;

 * sqlite:///chinook.db
Done.


Name
Dazed And Confused
Space Truckin'
Dazed And Confused
We've Got To Get Together/Jingo
Funky Piano
Going Down / Highway Star
Santana Jam
The Sun Road
Whole Lotta Love
Mistreated (Alternate Version)


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра ROCK, приобретенные сотрудниками Microsoft

In [58]:
%%sql
SELECT Composer, --имена композиторов(групп, как я понял)
       tracks.Name, --названия песен
       Title --названия альбомов
FROM
       tracks
LEFT JOIN (SELECT genres.Name as GenreName,
                  *
           FROM genres, albums ) as Genres_and_Albums USING (GenreId, AlbumID)
          ,(SELECT * FROM invoice_items) USING (TrackId)
          ,(SELECT * FROM invoices     ) USING (InvoiceId)
          ,(SELECT * FROM customers    ) USING (CustomerId)
WHERE  
      GenreName = 'Rock' AND 
      Company like "Microsoft%"

 * sqlite:///chinook.db
Done.


Composer,Name,Title
"Dave Grohl, Taylor Hawkins, Nate Mendel, Chris Shiflett",No Way Back,In Your Honor [Disc 1]
"Dave Grohl, Taylor Hawkins, Nate Mendel, Chris Shiflett/FOO FIGHTERS",Still,In Your Honor [Disc 2]
"Dave Grohl, Taylor Hawkins, Nate Mendel, Chris Shiflett/FOO FIGHTERS",Razor,In Your Honor [Disc 2]
"Dave, Taylor, Nate, Chris",My Hero,The Colour And The Shape
None,Double Talkin' Jive,Use Your Illusion I
None,The Garden,Use Your Illusion I
None,Don't Damn Me,Use Your Illusion I
None,Dead Horse,Use Your Illusion I


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек


In [59]:
%%sql
SELECT
      g.Name               AS genres_name
      ,mt.Name             AS mediaType_name
      ,AVG(UnitPrice)      AS average_UnitPrice
      ,COUNT(tracks.Name)  AS countsOfTracks
FROM
      tracks

INNER JOIN 
          (SELECT * FROM genres     ) AS g  USING (GenreId),
          (SELECT * FROM media_types) AS mt USING (MediaTypeId)

GROUP BY  GenreId
HAVING UnitPrice > 1.5

 * sqlite:///chinook.db
Done.


genres_name,mediaType_name,average_UnitPrice,countsOfTracks
Science Fiction,Protected MPEG-4 video file,1.9899999999999995,13
TV Shows,Protected MPEG-4 video file,1.990000000000001,93
Sci Fi & Fantasy,Protected MPEG-4 video file,1.9900000000000004,26
Drama,Protected MPEG-4 video file,1.9899999999999978,64
Comedy,Protected MPEG-4 video file,1.9899999999999995,17


## Задание 4
Вывести компании, сделавшие максимальное и минимальное число заказов. Необходимо исключить клиентов без компании (NULL)

In [60]:
%%sql
SELECT
       Company,
       COUNT(InvoiceId) AS count --кол-во купленной музыки
FROM
    customers c,
    invoices  i
WHERE
    c.CustomerId = i.CustomerId 
GROUP BY c.Company
HAVING (c.Company != "None")

 * sqlite:///chinook.db
Done.


Company,count
Apple Inc.,7
Banco do Brasil S.A.,7
Embraer - Empresa Brasileira de Aeronáutica S.A.,7
Google Inc.,7
JetBrains s.r.o.,7
Microsoft Corporation,7
Riotur,7
Rogers Canada,7
Telus,7
Woodstock Discos,7


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [61]:
%%sql
SELECT DISTINCT
    Company
    ,COUNT(t.Name)
FROM
    customers

LEFT JOIN (SELECT * FROM invoices)      USING (CustomerId),
          (SELECT * FROM invoice_items) USING (InvoiceId),
          (SELECT * FROM tracks) AS t   USING (TrackId),
          (SELECT * FROM genres) AS g   USING (GenreId)

WHERE g.Name  =  'Pop' AND 
      Company != "None"

GROUP BY Company

 * sqlite:///chinook.db
Done.


Company,COUNT(t.Name)
Embraer - Empresa Brasileira de Aeronáutica S.A.,2
JetBrains s.r.o.,4


## Задание 6
Для каждого артиста определить все жанры песен, которые у него есть и количество песен по каждому жанру

In [62]:
%%sql
SELECT
      a.Name
      ,GenreName
      ,COUNT(TrackId)
FROM 
      artists a
LEFT JOIN (SELECT * FROM albums) AS newAlbums USING (ArtistId),
          (SELECT * FROM tracks) AS newTracks USING (AlbumID),
          (SELECT Name AS GenreName, * FROM genres) USING (GenreId)
GROUP BY a.Name
         ,GenreName 

 * sqlite:///chinook.db
Done.


Name,GenreName,COUNT(TrackId)
AC/DC,Rock,18
Aaron Copland & London Symphony Orchestra,Classical,1
Aaron Goldberg,Jazz,1
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,2
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,Classical,1
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,1
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,1
Accept,Rock,4
Adrian Leaper & Doreen de Feis,Classical,1
Aerosmith,Rock,15


## Задание 7
Вывести средний размер альбома в байтах

In [63]:
%%sql
SELECT a.title
       ,AVG(t.Bytes)
FROM tracks t
     ,albums a 
WHERE t.albumId = a.albumId
GROUP BY a.title

 * sqlite:///chinook.db
Done.


Title,AVG(t.Bytes)
...And Justice For All,14344168.333333334
20th Century Masters - The Millennium Collection: The Best of Scorpions,4814130.916666667
"A Copland Celebration, Vol. I",3211245.0
A Matter of Life and Death,7203892.181818182
A Real Dead One,4785011.25
A Real Live One,5147947.454545454
A Soprano Inspired,5605648.0
A TempestadeTempestade Ou O Livro Dos Dias,8853899.066666666
A-Sides,9170782.529411765
Ace Of Spades,5948116.066666666


# Часть 2

## Задание 8
Вывести названия треков и их продолжительность, которые были приобретены компаниями, которые обслуживаются сотрудниками, нанятыми после 3 сентября 2002 года

In [64]:
%%sql
SELECT
      t.Name
     ,t.Milliseconds
FROM tracks t
INNER JOIN (SELECT TrackId,      InvoiceId  FROM invoice_items) AS ii USING (TrackId),
           (SELECT InvoiceId,    CustomerId FROM invoices)      AS i  USING (InvoiceId),
           (SELECT SupportRepId, CustomerId, 
                                 HireDate   FROM customers c,
                                                 (SELECT * FROM employees) e 
                                                  WHERE c.SupportRepId = e.EmployeeId
                                                              ) AS c USING (CustomerId)
WHERE HireDate > '2002-09-03 00:00:00'                                       

 * sqlite:///chinook.db
Done.


Name,Milliseconds
Put The Finger On You,205662
Inject The Venom,210834
Evil Walks,263497
Breaking The Rules,263288
Born To Move,342804
Brasil,243696
Coroné Antonio Bento,200437
Música Urbana 2,194899
Woman Is The Nigger Of The World (Ao Vivo),298919
Smells Like Teen Spirit (Ao Vivo),316865


## Задание 9
Для каждого артиста найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение общей суммы, заработанной артистом по всем другим жанрам к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, и оно является максимальным для артиста, то выбирается жанр раньше по алфавиту

In [65]:
%%sql 
SELECT 
      Artist_Name, --ИМЯ АРТИСТОВ
      Genre_Name,  --ИМЯ ЖАНРА
      SUM(UnitPrice) AS Total_Cash --ОБЩАЯ ПРИБЫЛЬ ЗА ОПРЕДЕЛЕННЫЙ ЖАНР
FROM
      tracks t

LEFT JOIN (SELECT Name AS Genre_Name,*   FROM genres       ) AS g  USING (GenreId ), --ДЛЯ ВЫВОДА НАЗВАНИЙ ЖАНРА
          (SELECT AlbumID, ArtistId      FROM albums       ) AS al USING (AlbumID ), --ДЛЯ ПОДКЛЮЧЕНИЯ ТАБЛИЦЫ artists
          (SELECT Name AS Artist_Name,*  FROM artists      ) AS ar USING (ArtistId), --ДЛЯ ВЫВОДА ИМЕН АРТИСТОВ
          (SELECT TrackId                FROM invoice_items) AS ii USING (TrackId )  --ДЛЯ ОПРЕДЕЛЕНИЯ ЖАНРА КУПЛЕННОГО ТРЕКА

GROUP BY Artist_Name
         ,Genre_Name

HAVING MAX(UnitPrice)

ORDER BY Artist_Name
         ,Total_Cash DESC

 * sqlite:///chinook.db
Done.


Artist_Name,Genre_Name,Total_Cash
AC/DC,Rock,15.840000000000002
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,0.99
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,0.99
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,1.98
Accept,Rock,4.95
Adrian Leaper & Doreen de Feis,Classical,0.99
Aerosmith,Rock,9.9
Alanis Morissette,Rock,7.920000000000001
Alice In Chains,Rock,6.930000000000001
Amy Winehouse,R&B/Soul,11.88


## Задание 10
Вывести альбом, который является минимальным по объему (в байтах) среди альбомов со стоимостью, большей средней цены за альбом по всем альбомам с таким же жанром (если у альбома песни с нескольким жанрами, то жанром является тот, у которого наибольшее количество песен в альбоме)

In [66]:
%%sql
SELECT Title
       ,Bytes
       ,albumId
FROM albums

LEFT JOIN  (SELECT * FROM tracks) AS t USING(AlbumId)
          ,(SELECT * FROM genres) AS g USING(GenreId)

GROUP BY Title
HAVING SUM(UnitPrice) > ( AVG(Unitprice) in (SELECT Unitprice FROM tracks t 
                                                    WHERE t.GenreId = GenreId))
ORDER BY SUM(Bytes) ASC
LIMIT 0,1 ---тут можно воспользоваться row_number()

 * sqlite:///chinook.db
Done.


Title,Bytes,AlbumId
English Renaissance,2256484,314
